In [1]:
import numpy as np
import matplotlib.pyplot as plt

计算复杂度不高，结果易于理解，对中间值的确实不敏感，可以处理不想干的特征。
确定：overfitting

In [116]:
from math import log

# 根据标签纸计算熵
# dataset是一个m*l的数组，最后一列是类别标签
def calShannonEnt(dataSet):
    counts = len(dataSet)
    labels={}
    for v in dataSet:
        className=v[-1]
        if className not in labels:
            labels[className]=0
        labels[className]+=1
        
    ent = 0
    for k in labels.keys():
        p= labels[k]*1.0/counts
        ent -= p*log(p,2)
        
    return ent

def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [117]:
dataSet,labels=createDataSet()
calShannonEnt(dataSet)

0.9709505944546686

In [118]:
# 根据指定的列索引和值，切分数据集并过滤对应的列
#dataset 数据集
## axis 选择的特征
def splitDs(dataSet, axis, value):
    re=[]
    for v in dataSet:
        if v[axis] == value:
            ## 去除当前属性
            tmp = v[:axis]
            tmp.extend(v[axis+1:])
            re.append(tmp)
    return re

In [119]:
splitDs(dataSet,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [120]:
## 决策树算法，选择最佳分类值
def chooseBestFeatureToSplit(dataSet):
    #获取属性数目
    counts = len(dataSet[0])-1
    ent=calShannonEnt(dataSet)
    
    #针对每个属性计算info gain
    gain=-100
    bastFeature=-1
    
    #遍历各个属性
    for i in range(counts):
        # 获取属性值
        fvalues=[v[i]  for v in dataSet]
        #排重
        fset=set(fvalues)
        newEnt=0
        
        #计算每个属性值的entropy
        for values in fset:
            #切分出指定的集合
            ds=splitDs(dataSet, i, values)
            newEnt -= len(ds)*1.0*calShannonEnt(ds)/counts
        
        if newEnt > gain:
            gain = newEnt
            bastFeature=i
    
    return ent+gain, bastFeature
        
chooseBestFeatureToSplit(dataSet)

(-0.40649315662706575, 0)

In [121]:
import operator
## 获取classList中的最大分类树
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount: classCount[vote]=0
        classCount[vote] += 1
    sortCount = sorted(classCount.iterms(), key=operator.itemgetter(1),reverse=True)
    
    return sortCount[0][0]


In [122]:
# m*(属性数量 + 值列) 
def createTree(dataSet,labels):
    ##完全相同则终止
    classList = [ v[-1] for v in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    ## 没有属性了则终止，返回最多的那个属性
    if len(dataSet[0]) == 1 :
        return majorityCnt(classList)
    
    #找到最好的feature，然后按照feature的值
    gain, bestFea=chooseBestFeatureToSplit(dataSet)
    bestFeatureName=labels[bestFea]
    
    vs =[v[bestFea] for v in dataSet]
    vs = set(vs)
    
    myTree = {bestFeatureName:{}}
    del(labels[bestFea])
    #开始切分
    for featureValue in vs:
        sublabels=labels[:]
        myTree[bestFeatureName][featureValue] = createTree( \
            splitDs(dataSet, bestFea, featureValue), sublabels)
        
    return myTree

In [193]:
createTree(dataSet,labels.copy())

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [194]:
#让分类器工作起来
def classify(inputTree, featLabels, testVec):
    print("===================",inputTree, featLabels)
    
    ## 如果是叶节点，则返回
    if isinstance(inputTree,str):
        return inputTree
    
    # 先获取根节点,属性名
    rootFeature = list(inputTree.keys())[0]
    # 计算改属性对应的idex
    idx=-1
        
    for i in range(len(featLabels)):
        if featLabels[i]==rootFeature: idx=i
                        
    # 开始分类
    judges=inputTree[rootFeature]
    testVecFeature = testVec[idx]
    
    for value in judges:
        if value == testVecFeature:
            return classify(inputTree[rootFeature][value],featLabels,testVec)

In [195]:
tree={'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
print(labels)
classify(tree,labels,[1,1])

['no surfacing', 'flippers']
=================== {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}} ['no surfacing', 'flippers']
=================== {'flippers': {0: 'no', 1: 'yes'}} ['no surfacing', 'flippers']
=================== yes ['no surfacing', 'flippers']


'yes'

In [182]:
def storeTree(inputTree,filename):
    import pickle
    fw=open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()

In [196]:
## 眼睛类型预测
fw=open('lenses.txt')
datas = [line.strip().split('\t') for line in fw.readlines()]
lensesLabels=['age','prescript','astigmatic','tearRate','result']
rs=createTree(datas,lensesLabels)
print(rs)

{'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}}}}


In [198]:
print( classify(rs,lensesLabels,['young','hyper','no','normal']))

=================== {'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}}}} ['age', 'prescript', 'astigmatic', 'result']
=================== {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}} ['age', 'prescript', 'astigmatic', 'result']
=================== {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}} ['age', 'prescript', 'astigmatic', 'result']
=================== soft ['age', 'prescript', 'astigmatic', 'result']
soft
